In [42]:
from preprocessing import preprocessing_csr
import collections
import math

In [43]:
class treeNode:
    """variables:
    name of the node, a count
    nodelink used to link similar items
    parent vaiable used to refer to the parent of the node in the tree
    node contains an empty dictionary for the children in the node"""
    
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = []

# get the child of this node with the name 'name'
    def get_child(self, name):
        for c in self.children:
            if c.name == name:
                return c
        
        return None
            
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur

#display tree in text. Useful for debugging        
    def disp(self, ind=1):
        print ('  '*ind, self.name, self.count)
        
        for child in self.children:
            child.disp(ind+1)

### Testin the Tree Structure

In [44]:
# testing the tree
# rootNode = treeNode('pyramid',9,None)
# rootNode.children.append(treeNode('eye',13,None))
# rootNode.disp()

## -----------------------

### Building the FP-tree

In [45]:
def updateHeader(nodeToTest, targetNode): 
    """This version does not use recursion."""
    
    # Do not use recursion to traverse a linked list!
    while (nodeToTest.nodeLink != None): 
        nodeToTest = nodeToTest.nodeLink
    
    nodeToTest.nodeLink = targetNode

In [46]:
def updateTree(items, inTree, headerTable, count):
    """grow the Fp-tree with an itemset"""
    
    # check if items[0] in retTree.children
    if inTree.get_child(items[0]) != None: 
        inTree.get_child(items[0]).inc(count)
    
    else:   #add items[0] to inTree.children
        inTree.children.append(treeNode(items[0], count, inTree))
        
        if headerTable[items[0]] == None: #update header table 
            #W A R N I N G!
            headerTable[items[0]] = inTree.children[
                len(inTree.children)-1] 
        else:
            updateHeader(headerTable[items[0]], 
                         inTree.children[len(inTree.children)-1])
    
    if len(items) > 1: #call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.get_child(items[0]), 
                   headerTable, count)

In [47]:
def createTree(dataSet): #create FP-tree from dataset but don't mine
    """takes the dataset and the minimum support as arguments and 
    builds the FP-tree. This makes two passes through the dataset. 
    The first pass goes through everything in the dataset and counts 
    the frequency of each term. These are stored in the header table."""
    
    headerTable = {}   
#     print 'dataset: ',len(dataSet)
    
    #go over dataSet twice
    for trans in dataSet:   #first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = None
    
#     print 'headerTable: ',headerTable

    itemSet = list(headerTable.keys())
#     print 'itemSet: ',itemSet
    
    for k in headerTable: # reformat headerTable to use Node link 
        headerTable[k] = None
    
#     print 'headerTable: ',headerTable

    retTree = treeNode('Null Set', 1, None) #create tree
    
    # dataSet.items() returns key-value tuple
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
            items = list(tranSet)
            # populate tree with ordered freq itemset
            updateTree(items, retTree, headerTable, count)
            
#     print 'headerTable: ',headerTable

    return retTree, headerTable

In [48]:
def loadSimpDat():
    """Get the data from preprocessing."""
    
    training, testing = preprocessing_csr.format(1)
#     simpDat = [['saan', 'pr', 'vb', 'abbreviation'], 
#                ['paano', 'pr', 'vb', 'dt', 'dt', 'nn', 'cc', 'pr', 'abbreviation'], 
#                ['ano', 'dt', 'nn', 'vb', 'cc', 'nn', 'pr', 'vb', 'nn', 'abbreviation'],
#                ['ano', 'pr', 'description'], 
#                ['anong', 'nn', 'pr', 'description'], 
#                ['bakit', 'dt', 'nn', 'pr', 'description'],
#                ['sino', 'rb', 'pr', 'entity'], 
#                ['alin', 'cc', 'cd', 'dt', 'vb', 'pr', 'entity'], 
#                ['alin', 'rb', 'pr', 'dt', 'nn', 'pr', 'entity'],
#                ['alin', 'dt', 'vb', 'entity'], 
#                ['sino', 'dt', 'nn', 'cc', 'nn', 'pr', 'human'], 
#                ['sino', 'rb', 'pr', 'human'], 
#                ['alin', 'cc', 'nn', 'dt', 'vb', 'pr', 'human'],
#                ['saan', 'rb', 'pr', 'rb', 'location'], 
#                ['saan', 'pr', 'rb', 'location'], 
#                ['nasaan', 'pr', 'rb', 'rb', 'location'],
#                ['kailan', 'rb', 'pr', 'vb', 'numeric'], 
#                ['kailan', 'pr', 'nn', 'cc', 'vb', 'vb', 'nn', 'numeric'], 
#                ['kailan', 'pr', 'vb', 'vb', 'numeric'], 
#                ['alin', 'cc', 'pr', 'cc', 'vb', 'cc', 'nn', 'dt', 'nn', 'pr', 'numeric'], 
#                ['ano', 'dt', 'nn', 'pr', 'vb', 'numeric'], 
#                ['paano', 'pr', 'vb', 'dt', 'dt', 'nn', 'cc', 'pr', 'abbreviation'], 
#                ['ano', 'dt', 'nn', 'vb', 'cc', 'nn', 'pr', 'vb', 'nn', 'abbreviation'],
#                ['ano', 'pr', 'description'], 
#                ['anong', 'nn', 'pr', 'description'], 
#                ['bakit', 'dt', 'nn', 'pr', 'description'],
#                ['sino', 'rb', 'pr', 'entity'], 
#                ['alin', 'cc', 'cd', 'dt', 'vb', 'pr', 'entity'], 
#                ['alin', 'rb', 'pr', 'dt', 'nn', 'pr', 'entity'],
#     ]
    return training, testing

In [49]:
def createInitSet(dataSet):
    """the createTree() function doesnt take the input data as lists. 
    It expectsa a dictionary with the itemsets as the dictionary keys 
    and the frequency as the value. A createInitSet() function does 
    this conversion for you."""

    retDict = collections.OrderedDict()

    for trans in dataSet:
        if tuple(trans) in retDict:
            retDict[tuple(trans)] += 1
        else:
            retDict[tuple(trans)] = 1

    return retDict

## -----------------------

### Testing the FP-tree

In [50]:
#testing the sample data
training, testing = loadSimpDat()
# print 'Training Data for Fp-growth algoithm:\n',training
print len(training)
# print '\nTesting Data: ',testing
print len(testing)

2462
616


In [51]:
initSet = createInitSet(training)
print initSet
print '\n', len(initSet)

OrderedDict([(('anong', 'jj', 'pr', 'vb', 'cc', 'vb', 'pr', 'cc', 'pr', 'dt', 'vb', 'pr', 'description'), 1), (('anong', 'vb', 'pr', 'description'), 1), (('anong', 'rb', 'pr', 'cc', 'rb', 'pr', 'description'), 1), (('bakit', 'rb', 'vb', 'pr', 'rb', 'description'), 1), (('bakit', 'pr', 'vb', 'rb', 'cc', 'pr', 'pr', 'vb', 'description'), 1), (('bakit', 'rb', 'rb', 'pr', 'cc', 'pr', 'description'), 2), (('paano', 'rb', 'rb', 'rb', 'pr', 'description'), 1), (('paano', 'rb', 'pr', 'vb', 'description'), 6), (('paano', 'rb', 'dt', 'nn', 'pr', 'vb', 'pr', 'description'), 1), (('kailan', 'pr', 'rb', 'vb', 'dt', 'vb', 'nn', 'numeric'), 3), (('kailan', 'rb', 'vb', 'dt', 'nn', 'nn', 'vb', 'cc', 'vb', 'numeric'), 1), (('kailan', 'rb', 'pr', 'nn', 'numeric'), 4), (('sino', 'dt', 'nn', 'cc', 'nn', 'pr', 'human'), 4), (('sino', 'rb', 'pr', 'human'), 4), (('sino', 'rb', 'pr', 'entity'), 3), (('alin', 'pr', 'dt', 'vb', 'pr', 'entity'), 13), (('alin', 'cc', 'nn', 'dt', 'vb', 'pr', 'human'), 2), (('alin',

In [52]:
#The FP-tree
myFPtree, myHeaderTab = createTree(initSet)
# print myHeaderTab
# myFPtree.disp()

## -----------------------

### Mine rules from the FP-tree and Training

In [53]:
def ascendTreeV2(leafNode, prefixPath):
    """ Ascends from leaf node to root, collecting the names of 
    items it encounters"""

    if leafNode.parent != None:
        prefixPath.append((leafNode.name, leafNode.count,))
        ascendTreeV2(leafNode.parent, prefixPath)

In [54]:
#treeNode comes from header table
def findPrefixPathV2(basePat, treeNode):
    """This function iterates through the linked list until it hits 
    the end. For each item it encounters, it calls ascendTree().
    This list is returned and added to the conditional category base 
    dictionary called condPats."""

    condPats = [] #list of all the paths with each category has
    
    while treeNode != None:
        prefixPath = []
        ascendTreeV2(treeNode, prefixPath)
#         print "prefixPath: ",  prefixPath
        condPats.append(list(reversed(list(prefixPath[1:]))))
            
        treeNode = treeNode.nodeLink
        
    return condPats

In [55]:
def mineTreeV2(myFPtree, headerTable, minSup):
    
    suffixes = ['entity', 'abbreviation', 'description', 'human', 
                'location', 'numeric']
    minedRules = []
    
    for i in range(0, len(suffixes)):
        condPattBases = findPrefixPathV2(suffixes[i], 
                                         headerTable[suffixes[i]])
        
#         print 'Conditional category Bases for: ' ,suffixes[i],
#         len(condPattBases), '\n', condPattBases
        
        condFpTree = []
        
        for p in range(0, len(condPattBases)):
            counts = [v[1] for v in condPattBases[p]]
            
            for q in range(0, len(counts)):
                if counts[q] < minSup:
                    break
                elif q == len(counts)-1:
                    condFpTree.append(condPattBases[p])
#                     element = tuple(t[0] for t in condPattBases[p])
#                     minedRules.append((element, suffixes[i],))
                    
        print 'Conditional Fp-Tree for: ' ,suffixes[i], 
        len(condFpTree), '\n', condFpTree
        
        # holds the frequent pattern of each suffix as array of tuples
        value = []
        
        for category in condFpTree:
            element = tuple(t[0] for t in category)
            value.append(element)
            
        minedRules.append([suffixes[i], value])
        
    return minedRules

In [56]:
rules = mineTreeV2(myFPtree, myHeaderTab, 80)

Conditional Fp-Tree for:  entity Conditional Fp-Tree for:  abbreviation Conditional Fp-Tree for:  description Conditional Fp-Tree for:  human Conditional Fp-Tree for:  location Conditional Fp-Tree for:  numeric


## -------------------------

### Conflict Resolution Schemes

In [57]:
def is_subset(rule, patt):
    r = list(rule)
    p = list(patt)
    
    if len(r) > len(p):
        return False
    else:
        l = p[:len(r)]
        if r == l:
            return True
        else:
            return False

In [58]:
def getOrderedRuleSet(trainingData, rules):
    """Rank all of the mined rules according to their accuracy.
    Form of both data: ((pattern), category)
    Returns the oredered rule set as an array of tuples in the
    form (((pattern), category), accuracy).
    """
    
#     trainingData = []
    
#     for data in trainingData: # converts training data to tuples
#         pat = tuple(data[:len(data)-1])
#         cat = data[len(data)-1]
#         trainingData.append((pat, cat,))
    
#     print "Local Training Data: ", "\n", trainingData
    
    antCounts = {} # number of occurances of the antecedent
    antConseqCounts = {}
    
    for r in rules: # populate dictionaries
        for pattern in r[1]:
            antCounts[pattern] = 0
            antConseqCounts[(pattern, r[0])] = 0
    
    # count number of antecedent in training data
    for data in trainingData: 
        if data[0] in antCounts:
            antCounts[data[0]] += 1
            
#     print "Antecedent Count Dictionary:\n", antCounts
    
    for data in trainingData:
        if data in antConseqCounts:
            antConseqCounts[data] += 1
            
#     print "Antecendent Consequence:\n", antConseqCounts
    
    orderedRuleSet = []
    for rule in antConseqCounts:
        accuracy = (float(antConseqCounts[rule])/
                    float(antCounts[rule[0]]))*100
        accuracy = int(accuracy)
        orderedRuleSet.append((rule, accuracy,))
        
    orderedRuleSet = sorted(orderedRuleSet, key=lambda rule: rule[1], 
                            reverse=True)
#     print "Ordered Rule Set:\n", orderedRuleSet
    
    return orderedRuleSet

In [59]:
def getDefaultClass(testData):
    """Data form: ((pattern), category)"""
    
    questWord = {'alin': 'entity', 'saan':'location', 'ano':'entity', 
                 'kailan':'numeric', 'paano':'description', 
                 'sino':'human', 'bakit':'description'}
    
    for key in questWord:
        if key in testData[0]:
            return questWord[key]
        
    return None

## -------------------------------

### Classification and Testing

In [60]:
def classify(testingData, rules, decisionList):
    """testingData form: ((pattern), category)
    rules form: [['category', [(pattern1), (pattern2)]]]
    decisionList form: [(((pattern), 'category'), accuracy)]"""

    
    table = []
    
    for data in testingData: # populate table
        row = []
        row.append(data[0])
        
        cat = []
        for r in rules: # question classification
            for category in r[1]:
                if data[0] == category:
                    cat.append(r[0])
        
        # Conflict Resolution
        # if sentence structure did not trigger any category
        if len(cat) == 0: 
            for i in range(0, len(rules)): # for each category
                # for each pattern in that category
                for j in range(0, len(rules[i][1])): 
                    # check if any rule is a subset of a sentence
                    if is_subset(rules[i][1][j], data[0]) == True:
                        cat.append(rules[i][0])
                        break
                    # if testing still didn't trigger any rule
                    # assign default class according to its question word
                    elif j == len(rules)-1 and is_subset(rules[i][1][j], 
                                                         data[0])==False:
                        default = getDefaultClass(data[0])
                        if default != None:
#                             print "Helloooooooo"
                            cat.append(default)
                            break
            if len(cat)>1: # more than one category was triggered
                default = getDefaultClass(data[0])
                if default != None:
                    cat = []
                    cat.append(default)

#   For debugging purpose:                  
            if len(cat) > 1:
                print "cat=0: ", data[0], cat
        elif len(cat) > 1:
            ranks = []
            # for each category triggered by data
            for i in range(0, len(cat)): 
                # combine data pattern with category as tuples
                tmpD = (data[0], cat[i],) 
                
                # for each rule in decisionList
                for rule in decisionList:
                    # if combined pattern and category match rule[i]
                    if tmpD == rule[0]: 
                        print rule[0], tmpD
                        # get the index
                        ranks.append(decisionList.index(rule))
#                         print ranks
            
            newCat = []
            newCat.append(cat[ranks.index(max(ranks))])
            cat = newCat
            
#           For debugging purpose:                  
#             if len(cat):
#                 print "cat=1: ", data[0], cat
            
        row.append(cat)
        table.append(row)
    
    return table

In [61]:
def initTestingSet(testingSet):
    retSet = []
    
    for data in testingSet:
        key = tuple(data[:len(data)-2])
        value = data[len(data)-1]
        retSet.append((key,value,))
    
    return retSet

In [62]:
def initTrainingData(trainingSet):
    """trainingSet form: ['anong', 'jj', 'pr', 'vb', 'cc', 
    'vb', 'pr', 'cc', 'pr', 'dt', 'vb', 'pr', 'description']
    Returns retSet in the form: ((pattern), category)"""
    
    retSet = []
    
    for data in trainingSet:
        lst = list(data)
        pattern = tuple(lst[:len(lst)-1])
        category = lst[len(lst)-1]
        retSet.append((pattern, category,))
            
    return retSet

In [63]:
testingData = initTestingSet(testing)
print len(testingData)
print len(testing)

trainingData = initTrainingData(training)
print len(trainingData)
print len(training)

616
616
2462
2462


In [64]:
decisionList = getOrderedRuleSet(trainingData, rules)
table = classify(testingData, rules, decisionList)
# print testingData
# print "\n\nTable: ", table

(('saan', 'pr'), 'description') (('saan', 'pr'), 'description')
(('saan', 'pr'), 'location') (('saan', 'pr'), 'location')
(('kailan', 'pr', 'vb'), 'description') (('kailan', 'pr', 'vb'), 'description')
(('kailan', 'pr', 'vb'), 'numeric') (('kailan', 'pr', 'vb'), 'numeric')
(('saan', 'pr'), 'description') (('saan', 'pr'), 'description')
(('saan', 'pr'), 'location') (('saan', 'pr'), 'location')
(('paano', 'pr', 'vb'), 'entity') (('paano', 'pr', 'vb'), 'entity')
(('paano', 'pr', 'vb'), 'description') (('paano', 'pr', 'vb'), 'description')
(('paano', 'pr', 'vb'), 'human') (('paano', 'pr', 'vb'), 'human')
(('saan', 'pr'), 'description') (('saan', 'pr'), 'description')
(('saan', 'pr'), 'location') (('saan', 'pr'), 'location')
(('saan', 'pr'), 'description') (('saan', 'pr'), 'description')
(('saan', 'pr'), 'location') (('saan', 'pr'), 'location')
(('saan', 'pr'), 'description') (('saan', 'pr'), 'description')
(('saan', 'pr'), 'location') (('saan', 'pr'), 'location')
(('saan', 'pr'), 'descript

## ---------------------------

### Performance Evaluation

In [65]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score

In [66]:
def measure_perf(testingData, table):
    perfTable = []
    
    for i in range(0, len(testingData)):
        row = []
        row.append(testingData[i][1])
        row.append(table[i][1])
        perfTable.append(row)
    
#     print perfTable
    
    correct = 0
    misclassified = 0
    unclassified = 0
    multCat = 0
    
    for item in perfTable:
        if len(item[1]) > 1:
            multCat += 1
        else:
            if item[0] == item[1][0]:
                correct += 1
            elif len(item[1]) == 0:
                unclassified += 1
            elif item[0] != item[1][0]:
                misclassified += 1
            
    print "Total number of testing data: ", len(testingData)
    print "Correctly classified items: ", correct
    print "Misclassified items: ", misclassified
    print "Unclassified items: ", unclassified
    print "Mutiple Classified items: ", multCat
    
    return perfTable

In [67]:
assignedPredTab = measure_perf(testingData, table)

Total number of testing data:  616
Correctly classified items:  485
Misclassified items:  131
Unclassified items:  0
Mutiple Classified items:  0


In [70]:
category = {'entity':1, 'abbreviation':2, 'description':3, 'human':4, 
            'location':5, 'numeric':6}

y_true = []
y_pred = []

for row in assignedPredTab:
    y_true.append(category[row[0]])
    y_pred.append(category[row[1][0]])

precision_recall_fscore_support(y_true, y_pred, average='macro')

print y_pred

[4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 6, 6, 3, 4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 6, 6, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 1, 3, 6, 6, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 6, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 6, 6, 3, 4, 4, 4, 1, 1, 1, 5, 5, 5, 3, 3, 3, 6, 4, 1, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 6, 6, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 3, 6, 6, 6, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 6, 6, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 6, 6, 4, 4, 4, 1, 5, 5, 5, 3, 1, 1, 3, 3, 3, 3, 3, 3, 6, 6, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 1, 6, 6, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 1, 1, 1, 3, 3, 3, 3, 3, 3, 6, 6, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 3, 1, 1, 3, 3, 3, 3, 3, 3, 6, 3, 6, 4, 4, 4, 1, 1, 1, 5, 5, 5, 3, 1, 1, 3, 3, 3, 3, 3, 3, 6, 6, 

In [71]:
# calculate f-score for each category
f1_score(y_true,y_pred, average=None)

array([ 0.69172932,  0.        ,  0.77161863,  0.82608696,  0.95505618,
        0.80555556])

In [72]:
# calculate f-score of the classifier
f1_score(y_true,y_pred, average='weighted')

0.78695511536841034